# Importing packages

In [8]:
from __future__ import print_function
from __future__ import division
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import cv2
import csv
import sys
import os
from datetime import datetime
print("Done!")


Done!


# Set default tensors to GPU tensors

In [9]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.cuda.set_device(0)
dtype = torch.cuda.FloatTensor
print("Current Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
print("Current Device ID:",torch.cuda.current_device(), "Device to use:",torch.cuda.device(0))
print("Total devices count:", torch.cuda.device_count())
print("Done!")

Current Device Name: GeForce GTX 1080 Ti
Current Device ID: 0 Device to use: <torch.cuda.device object at 0x000002B103444BA8>
Total devices count: 1
Done!


# Defining dataset

In [10]:
class Dataset(data.Dataset):
    def __init__(self, samples, image_dir, transform=None):
        self.samples = samples
        self.image_dir = image_dir
        self.transform = transform
    
    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])
        center_img, steering_angle_center = augment(self.image_dir, batch_samples[0], steering_angle)
        left_img, steering_angle_left = augment(self.image_dir, batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right = augment(self.image_dir, batch_samples[2], steering_angle - 0.4)
        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)
        return (center_img, float(steering_angle_center)),\
               (left_img, float(steering_angle_left)),\
               (right_img, float(steering_angle_right))
               
    def __len__(self):
        return len(self.samples)
print("Done!")

Done!


# Defining Model

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(87040, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        num_features=self.num_flat_features(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
print("Done!")

Done!


# defining functions

In [12]:
def read_samples(csv_filepath, validation_per = 0.2):
    samples = []
    with open(csv_filepath) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for line in reader:                                                                 
            samples.append(line)
    validation_count = int(validation_per * len(samples))
    training_count = len(samples) - validation_count
    training_samples, validation_samples = random_split(samples,\
                                                        lengths = [training_count, validation_count])
    return training_samples, validation_samples 

def augment(image_dir, imgName, angle):
    name = image_dir + imgName.split('/')[-1]
    current_image = cv2.imread(name)
    current_image = current_image[65:-25, :, :]
    if np.random.rand() < 0.5:
        current_image = cv2.flip(current_image, 1)
        angle = angle * -1.0  
    return current_image, angle

def eval_net(dataloader):
    correct = 0
    total = 0
    total_loss = 0
    net.eval() # Why would I do this?
    criterion = nn.MSELoss(reduction='sum')
    for center, left, right in dataloader:
        for inputs, targets in [center, left, right]:
            inputs, targets = Variable(inputs).cuda(),\
                              Variable(targets.float()).cuda().unsqueeze(1)
            outputs = net(inputs)
            total += targets.size(0)
            loss = criterion(outputs, targets)
            total_loss += loss.data.item()
    net.train() # Why would I do this?
    return total_loss / total
print("Done!")

Done!


# Training

In [14]:
if __name__ == "__main__":
    BATCH_SIZE = 32 #mini_batch size
    MAX_EPOCH = 100  #maximum epoch to train
   
    data_dir = "./data/"

    train_samples, test_samples = read_samples(data_dir + 'driving_log.csv')                     
     
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = Dataset(train_samples, data_dir + 'IMG/', transform)
    test_set = Dataset(test_samples, data_dir + 'IMG/', transform)

    trainloader = DataLoader(train_set,\
                             batch_size=BATCH_SIZE,\
                             shuffle=True,\
                             num_workers=0)
    testloader = DataLoader(test_set,\
                            batch_size=BATCH_SIZE,\
                            shuffle=False,\
                            num_workers=0)


    print('Building model...')
    net = Net().cuda()
    net.train() # Why would I do this?

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    train_losses=list()
    test_losses=list()

    print('Start training...')
    for epoch in range(MAX_EPOCH):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, (center, left, right) in enumerate(trainloader, 0):
            for inputs, targets in [center, left, right]:
                # get the inputs
                inputs, targets = center # only use center for now
                # wrap them in Variable
                inputs, targets = Variable(inputs).cuda(),\
                                  Variable(targets.float()).cuda().unsqueeze(1)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = net(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                # print statistics
                running_loss += loss.data.item()
            if i % 500 == 499:    # print every 2000 mini-batches
                print('    Step: %5d avg_batch_loss: %.5f' %
                      (i + 1, running_loss / 500))
                running_loss = 0.0
        print('    Finish training this EPOCH, start evaluating...')
        train_loss = eval_net(trainloader)
        test_loss = eval_net(testloader)
        print('EPOCH: %d train_loss: %.5f test_loss: %.5f' %
              (epoch+1, train_loss, test_loss))
        train_losses.append(train_loss)
        test_losses.append(test_loss)
    print('Finished Training')
print("Done!")

Building model...
Start training...
    Finish training this EPOCH, start evaluating...
EPOCH: 1 train_loss: 0.09558 test_loss: 0.09591
    Finish training this EPOCH, start evaluating...
EPOCH: 2 train_loss: 0.09294 test_loss: 0.09343
    Finish training this EPOCH, start evaluating...
EPOCH: 3 train_loss: 0.09428 test_loss: 0.09527
    Finish training this EPOCH, start evaluating...
EPOCH: 4 train_loss: 0.09583 test_loss: 0.09661
    Finish training this EPOCH, start evaluating...
EPOCH: 5 train_loss: 0.09706 test_loss: 0.09812
    Finish training this EPOCH, start evaluating...
EPOCH: 6 train_loss: 0.08976 test_loss: 0.09047
    Finish training this EPOCH, start evaluating...
EPOCH: 7 train_loss: 0.09537 test_loss: 0.09657
    Finish training this EPOCH, start evaluating...
EPOCH: 8 train_loss: 0.09293 test_loss: 0.09402
    Finish training this EPOCH, start evaluating...
EPOCH: 9 train_loss: 0.09395 test_loss: 0.09507
    Finish training this EPOCH, start evaluating...
EPOCH: 10 tr

EPOCH: 81 train_loss: 0.08202 test_loss: 0.08447
    Finish training this EPOCH, start evaluating...
EPOCH: 82 train_loss: 0.08195 test_loss: 0.08444
    Finish training this EPOCH, start evaluating...
EPOCH: 83 train_loss: 0.08201 test_loss: 0.08460
    Finish training this EPOCH, start evaluating...
EPOCH: 84 train_loss: 0.08176 test_loss: 0.08440
    Finish training this EPOCH, start evaluating...
EPOCH: 85 train_loss: 0.08184 test_loss: 0.08441
    Finish training this EPOCH, start evaluating...
EPOCH: 86 train_loss: 0.08199 test_loss: 0.08463
    Finish training this EPOCH, start evaluating...
EPOCH: 87 train_loss: 0.08192 test_loss: 0.08460
    Finish training this EPOCH, start evaluating...
EPOCH: 88 train_loss: 0.08195 test_loss: 0.08466
    Finish training this EPOCH, start evaluating...
EPOCH: 89 train_loss: 0.08194 test_loss: 0.08436
    Finish training this EPOCH, start evaluating...
EPOCH: 90 train_loss: 0.08180 test_loss: 0.08436
    Finish training this EPOCH, start eval

# saving the model

In [15]:
now_str = str(datetime.now())[:10]
plt.plot(train_losses, label='train_loss')
plt.plot(test_losses, label='test_loss')
plt.legend()
name = "./models/model_" + str(now_str) + "_100_new_Epoch_" + str(min(test_losses))
plt.savefig(name +".png")
print('Saving model...')
torch.save(net.state_dict(), name+".pth")
print("Done!")

Saving model...
Done!
